In [3]:
import os
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForQuestionAnswering,
    CLIPModel,
    CLIPProcessor
)
from PIL import Image
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.chunk import RegexpParser
from collections import Counter
import numpy as np
import docx
import io

class DocumentAnalyzer:
    def __init__(self):
        # Expanded NLTK resources list
        nltk_resources = [
            'punkt',
            'averaged_perceptron_tagger',
            'maxent_ne_chunker',
            'words',
            'stopwords',
            'tokenizers/punkt/PY3/english.pickle'
        ]
        
        # Create NLTK data directory if it doesn't exist
        nltk_data_dir = os.path.expanduser('~/nltk_data')
        os.makedirs(nltk_data_dir, exist_ok=True)
        
        # Download NLTK data with better error handling
        for resource in nltk_resources:
            try:
                nltk.download(resource, quiet=True, raise_on_error=True)
            except Exception as e:
                print(f"Warning: Issue downloading {resource}: {str(e)}")
                try:
                    # Fallback: try downloading to user directory
                    nltk.download(resource, download_dir=nltk_data_dir, quiet=True)
                except Exception as e2:
                    print(f"Error: Could not download {resource} to user directory: {str(e2)}")

        # Verify critical resources
        try:
            # Test tokenization
            test_text = "This is a test sentence."
            tokens = word_tokenize(test_text)
            if not tokens:
                raise Exception("Tokenization failed")
            
            # Test stopwords
            stops = stopwords.words('english')
            if not stops:
                raise Exception("Stopwords not available")
                
        except Exception as e:
            print(f"Critical NLTK components missing: {str(e)}")
            print("Please run the following commands manually:")
            print("import nltk")
            print("nltk.download('punkt')")
            print("nltk.download('stopwords')")
            raise

        # Initialize models with error handling
        try:
            self.qa_tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
            self.qa_model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")
            
            self.vision_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
            self.vision_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            
        except Exception as e:
            print(f"Error initializing models: {str(e)}")
            raise

        # Define grammar for extracting concepts
        self.chunk_grammar = "NP: {<DT>?<JJ>*<NN.*>+}"
        self.chunker = RegexpParser(self.chunk_grammar)

    def extract_text_from_word(self, doc_path):
        """Extract text from Word document"""
        try:
            doc = docx.Document(doc_path)
            full_text = []
            
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():  # Only add non-empty paragraphs
                    full_text.append(paragraph.text)
            
            return '\n'.join(full_text)
        except Exception as e:
            print(f"Error reading Word document: {str(e)}")
            return ""

    def extract_images_from_word(self, doc_path):
        """Extract images from Word document"""
        try:
            doc = docx.Document(doc_path)
            images = []
            
            for rel in doc.part.rels.values():
                if "image" in rel.target_ref:
                    image_data = rel.target_part.blob
                    image = Image.open(io.BytesIO(image_data))
                    images.append(image)
            
            return images
        except Exception as e:
            print(f"Error extracting images: {str(e)}")
            return []

    def find_main_topics(self, text):
        """Extract main topics from text with improved error handling"""
        try:
            # Ensure text is not empty
            if not text.strip():
                return []

            # Tokenize with explicit error handling
            try:
                words = word_tokenize(text.lower())
            except Exception as e:
                print(f"Tokenization error: {str(e)}")
                words = text.lower().split()  # Fallback to basic splitting

            # Get stopwords with fallback
            try:
                stop_words = set(stopwords.words('english'))
            except Exception as e:
                print(f"Stopwords error: {str(e)}")
                # Basic fallback stopwords
                stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}

            # Filter words
            words = [w for w in words if w.isalnum() and w not in stop_words]
            
            # Get word frequency
            word_freq = Counter(words)
            
            # Find important phrases with error handling
            topics = []
            try:
                sentences = sent_tokenize(text)
            except Exception as e:
                print(f"Sentence tokenization error: {str(e)}")
                sentences = text.split('.')  # Fallback to basic splitting
                
            for sentence in sentences:
                try:
                    tokens = nltk.pos_tag(word_tokenize(sentence))
                    chunks = self.chunker.parse(tokens)
                    
                    for subtree in chunks.subtrees():
                        if subtree.label() == 'NP':
                            topic = ' '.join(word for word, tag in subtree.leaves())
                            if topic.strip():  # Only add non-empty topics
                                topics.append(topic)
                except Exception as e:
                    print(f"Chunking error for sentence: {str(e)}")
                    continue
            
            # Score topics
            topic_scores = {}
            for topic in set(topics):
                words_in_topic = topic.lower().split()
                score = sum(word_freq.get(word, 0) for word in words_in_topic)
                topic_scores[topic] = score
            
            # Return top topics, or basic word frequency if no topics found
            if topic_scores:
                return sorted(topic_scores.items(), key=lambda x: x[1], reverse=True)[:5]
            else:
                # Fallback to most common words
                return [(word, freq) for word, freq in word_freq.most_common(5)]
            
        except Exception as e:
            print(f"Error in topic extraction: {str(e)}")
            return []

    def analyze_image_content(self, image):
        """Analyze what's in the image"""
        try:
            inputs = self.vision_processor(images=image, return_tensors="pt")
            
            # Simple categories for beginners
            categories = [
                "diagram", "chart", "photo", "drawing", "graph",
                "table", "map", "illustration"
            ]
            
            with torch.no_grad():
                image_features = self.vision_model.get_image_features(**inputs)
                text_inputs = self.vision_processor(
                    text=categories,
                    return_tensors="pt",
                    padding=True
                )
                text_features = self.vision_model.get_text_features(**text_inputs)
                
                similarity = torch.nn.functional.cosine_similarity(
                    image_features.unsqueeze(1),
                    text_features,
                    dim=-1
                )
                
                probs = torch.nn.functional.softmax(similarity, dim=-1)
                
                # Get top matches
                top_matches = [(categories[i], probs[0][i].item())
                              for i in range(len(categories))]
                top_matches.sort(key=lambda x: x[1], reverse=True)
                
                return top_matches[:2]
                
        except Exception as e:
            print(f"Error analyzing image: {str(e)}")
            return []

    def create_questions(self, text, topics, image_types=None):
        """Create simple, beginner-friendly questions"""
        try:
            questions = []
            
            # Basic question templates
            basic_templates = [
                "What is {}?",
                "Can you explain {}?",
                "Why is {} important?",
                "How does {} work?",
                "What are the main parts of {}?"
            ]
            
            # Create questions about main topics
            for topic, _ in topics[:3]:  # Use top 3 topics
                template = np.random.choice(basic_templates)
                question = template.format(topic)
                questions.append(question)
            
            # Add questions about images if present
            if image_types:
                image_templates = [
                    "What does the {} show about the topic?",
                    "How does the {} help explain the content?",
                    "What information can we learn from the {}?"
                ]
                
                for img_type, _ in image_types:
                    template = np.random.choice(image_templates)
                    question = template.format(img_type)
                    questions.append(question)
            
            return list(set(questions))  # Remove any duplicates
            
        except Exception as e:
            print(f"Error creating questions: {str(e)}")
            return []

    def analyze_word_document(self, doc_path):
        """Main function to analyze Word document"""
        try:
            results = {
                'text': '',
                'topics': [],
                'images': [],
                'questions': []
            }
            
            # Get text content
            text = self.extract_text_from_word(doc_path)
            results['text'] = text
            
            # Find main topics
            topics = self.find_main_topics(text)
            results['topics'] = topics
            
            # Process images
            images = self.extract_images_from_word(doc_path)
            image_analyses = []
            for image in images:
                image_type = self.analyze_image_content(image)
                if image_type:
                    image_analyses.append(image_type[0])  # Use top match
            results['images'] = image_analyses
            
            # Generate questions
            questions = self.create_questions(text, topics, image_analyses)
            results['questions'] = questions
            
            return results
            
        except Exception as e:
            print(f"Error analyzing document: {str(e)}")
            return {
                'text': '',
                'topics': [],
                'images': [],
                'questions': [],
                'error': str(e)
            }

# Example usage
def main():
    try:
        # Initialize analyzer
        analyzer = DocumentAnalyzer()
        
        # Analyze Word document
        doc_path = "/Users/vyakaranamsowmya/Desktop/Bot-LLM-1/Electromagnetic Induction- Class 9.docx"  # Replace with actual path
        results = analyzer.analyze_word_document(doc_path)
        
        # Print results in a user-friendly way
        print("\n📑 Document Analysis Results:")
        
        print("\n📌 Main Topics Found:")
        for topic, score in results['topics']:
            print(f"- {topic}")
        
        print("\n🖼️ Images Found:")
        for img_type, confidence in results['images']:
            print(f"- Found a {img_type}")
        
        print("\n❓ Generated Questions:")
        for i, question in enumerate(results['questions'], 1):
            print(f"{i}. {question}")
            
    except Exception as e:
        print(f"❌ Error running analysis: {str(e)}")

if __name__ == "__main__":
    main()

Critical NLTK components missing: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/Users/vyakaranamsowmya/nltk_data'
    - '/Users/vyakaranamsowmya/Desktop/Bot-LLM-1/venv/nltk_data'
    - '/Users/vyakaranamsowmya/Desktop/Bot-LLM-1/venv/share/nltk_data'
    - '/Users/vyakaranamsowmya/Desktop/Bot-LLM-1/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Please run the following commands manually:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
❌ Error running analysis: 
************************************************

[nltk_data] Error loading tokenizers/punkt/PY3/english.pickle: Package
[nltk_data]     'tokenizers/punkt/PY3/english.pickle' not found in
[nltk_data]     index
[nltk_data] Error loading tokenizers/punkt/PY3/english.pickle: Package
[nltk_data]     'tokenizers/punkt/PY3/english.pickle' not found in
[nltk_data]     index


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import trimesh
import pyrender
import os
from PIL import Image
import json
from torchvision import transforms

def install_requirements():
    """
    Install required packages using pip
    """
    import subprocess
    packages = [
        'trimesh',
        'pyrender',
        'torch',
        'torchvision',
        'pillow',
        'numpy',
    ]
    for package in packages:
        subprocess.check_call(['pip', 'install', package])

class ModelRenderer:
    def __init__(self):
        self.scene = pyrender.Scene()
        
    def setup_camera(self):
        """Setup camera for rendering"""
        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
        self.camera_node = pyrender.Node(camera=camera)
        self.scene.add_node(self.camera_node)
        
    def render_viewpoints(self, mesh_path, num_views=8, output_dir="renders"):
        """Render model from different viewpoints using pyrender"""
        os.makedirs(output_dir, exist_ok=True)
        images = []
        
        # Load mesh using trimesh and convert to pyrender
        mesh = trimesh.load_mesh(mesh_path)
        if isinstance(mesh, trimesh.Scene):
            mesh = mesh.dump(concatenate=True)  # Merge all geometries into a single mesh
        mesh = pyrender.Mesh.from_trimesh(mesh)
        mesh_node = pyrender.Node(mesh=mesh)
        self.scene.add_node(mesh_node)
        
        # Add light
        light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=5.0)
        light_node = pyrender.Node(light=light)
        self.scene.add_node(light_node)
        
        r = pyrender.OffscreenRenderer(viewport_width=640, viewport_height=480)
        
        for i in range(num_views):
            angle = (2.0 * np.pi * i) / num_views
            radius = 2.0  # Distance from center
            
            # Position camera
            cam_pos = np.array([
                radius * np.cos(angle),
                radius * np.sin(angle),
                1.0
            ])
            
            # Look at center
            cam_target = np.array([0.0, 0.0, 0.0])
            cam_up = np.array([0.0, 0.0, 1.0])
            
            # Compute camera matrix
            cam_matrix = look_at(cam_pos, cam_target, cam_up)
            self.scene.set_pose(self.camera_node, cam_matrix)
            
            # Render
            color, depth = r.render(self.scene)
            image = Image.fromarray(color)
            
            output_path = os.path.join(output_dir, f"view_{i}.png")
            image.save(output_path)
            images.append(output_path)
            
        r.delete()
        return images

def look_at(pos, target, up):
    """Create look-at matrix for camera"""
    forward = np.array(target) - np.array(pos)
    forward = forward / np.linalg.norm(forward)
    
    right = np.cross(forward, up)
    right = right / np.linalg.norm(right)
    
    new_up = np.cross(right, forward)
    new_up = new_up / np.linalg.norm(new_up)
    
    mat = np.eye(4)
    mat[:3, 0] = right
    mat[:3, 1] = new_up
    mat[:3, 2] = -forward
    mat[:3, 3] = pos
    
    return mat

class Model3DDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
        
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

class PartDetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(PartDetectionModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
    
    return model

def main(mesh_path, output_dir):
    # Render views
    renderer = ModelRenderer()
    renderer.setup_camera()
    image_paths = renderer.render_viewpoints(mesh_path, output_dir=output_dir)
    
    # Prepare dataset
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Assuming you have labels for the parts
    labels = {"part1": 0, "part2": 1}  # Replace with actual labels
    
    dataset = Model3DDataset(image_paths, labels, transform=transform)
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Train model
    model = PartDetectionModel(num_classes=len(set(labels.values())))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    trained_model = train_model(model, train_loader, criterion, optimizer)
    
    # Save model and labels
    torch.save(trained_model.state_dict(), os.path.join(output_dir, 'model.pth'))
    with open(os.path.join(output_dir, 'labels.json'), 'w') as f:
        json.dump(labels, f)

if __name__ == "__main__":
    # Install required packages
    install_requirements()
    
    mesh_path = "/Users/vyakaranamsowmya/Desktop/Bot-LLM-1/power generation.obj"  # Replace with your OBJ file path
    output_dir = "output"
    main(mesh_path, output_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TypeError: Expected a Trimesh or a list, got a <class 'trimesh.scene.scene.Scene'>